<a href="https://colab.research.google.com/github/ShaswataJash/LargeDatasetHandling/blob/master/Incremental_min_max_calculation_for_large_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!uname -a
!python --version

Linux 8591b8ca8e76 5.10.147+ #1 SMP Sat Dec 10 16:00:40 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
Python 3.9.16


In [1]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:                        3
CPU MHz:                         2000.180
BogoMIPS:                        4000.36
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        1 MiB
L3 cache:                        38.5 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

In [3]:
import torch
print(torch.__version__)

2.0.0+cu118


In [ ]:
!df -h /dev/shm

Filesystem      Size  Used Avail Use% Mounted on
shm             5.7G     0  5.7G   0% /dev/shm


In [ ]:
#https://stackoverflow.com/questions/7878707/how-to-unmount-a-busy-device
#for python multipprocessor, data across child process and main process are being shared through shared memory
#for pytorch Dataloader, shared memory requirement can be quite high
!sudo umount -l /dev/shm/ && sudo mount -t tmpfs -o rw,nosuid,nodev,noexec,relatime,size=9G shm /dev/shm

In [4]:
#refer: https://numpy.org/doc/stable/reference/global_state.html#madvise-hugepage-on-linux
!cat /sys/kernel/mm/transparent_hugepage/enabled
!cat /sys/kernel/mm/transparent_hugepage/defrag
!cat /sys/kernel/mm/transparent_hugepage/use_zero_page
!cat /sys/kernel/mm/transparent_hugepage/hpage_pmd_size

always [madvise] never
always defer defer+madvise [madvise] never
1
2097152


In [3]:
%env

{'SHELL': '/bin/bash',
 'NV_LIBCUBLAS_VERSION': '11.11.3.6-1',
 'NVIDIA_VISIBLE_DEVICES': 'all',
 'COLAB_JUPYTER_TRANSPORT': 'ipc',
 'NV_NVML_DEV_VERSION': '11.8.86-1',
 'NV_CUDNN_PACKAGE_NAME': 'libcudnn8',
 'CGROUP_MEMORY_EVENTS': '/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events',
 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.16.2-1+cuda11.8',
 'NV_LIBNCCL_DEV_PACKAGE_VERSION': '2.16.2-1',
 'VM_GCE_METADATA_HOST': '169.254.169.253',
 'HOSTNAME': '25fd51ec9744',
 'TBE_RUNTIME_ADDR': '172.28.0.1:8011',
 'GCE_METADATA_TIMEOUT': '3',
 'NVIDIA_REQUIRE_CUDA': 'cuda>=11.8 brand=tesla,driver>=450,driver<451 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,driver<471 brand=geforcertx,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=titan,driver>=470,driver<471 brand=titanrtx,driver>=470,driv

In [2]:
#https://stackoverflow.com/questions/37890898/how-to-set-env-variable-in-jupyter-notebook
%env NUMPY_MADVISE_HUGEPAGE=1

env: NUMPY_MADVISE_HUGEPAGE=1


#Determine total availiable GPU memory

In [1]:
#ref: https://stackoverflow.com/questions/59567226/how-to-programmatically-determine-available-gpu-memory-with-tensorflow
import subprocess as sp
import os
def get_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    try:
        memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
        memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
        return memory_free_values[0] * 1024 * 1024 # memory_free_values[0] is in MB, thus converting into bytes
    except Exception as e:
        print(e)
        return -1

#downloading kaggle competitions files

In [ ]:
!pip install kaggle==1.5.12

In [ ]:
%%python

import sys
import logging
import os
import subprocess

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s:%(levelname)s:%(message)s')
logger = logging.getLogger('my_logger')
#handling of kaggle interaction
try:
    os.environ["KAGGLE_CONFIG_DIR"] = '/home' #kaggle.json file should be uploaded to /home location before executing this cell
    kaggle_write_cmd = "kaggle competitions download -c open-problems-multimodal"
    kaggle_write_call = subprocess.run(kaggle_write_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    logger.info(kaggle_write_call.stdout)
    if kaggle_write_call.returncode != 0:
        logger.error("Error in kaggle download, errorcode=%s", kaggle_write_call.returncode)
        sys.stdout.flush()
        sys.exit("Forceful exit as kaggle download returned error")
except BaseException as err:
    logger.error("kaggle download related error", exc_info=True)
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while kaggle download")

In [ ]:
!mkdir /content/drive/MyDrive/colab_exp_result/kaggle_data
!unzip /content/open-problems-multimodal.zip -d /content/drive/MyDrive/colab_exp_result/kaggle_data

We can mount Google drive in colab and can copy the kaggle competitions files there. This will help not to run kaggle download code everytime before start of the notebook - it can save lot of time. Instead, everytime we can directly copy the contents from drive into the local filesystem of the underneath VM hosting the notebook.

In [5]:
!nohup cp /content/drive/MyDrive/colab_exp_result/kaggle_data/* /mnt &

nohup: appending output to 'nohup.out'


In [9]:
!ls -l /mnt

total 28181076
-rw------- 1 root root  2418406934 Apr 22 02:06 evaluation_ids.csv
-rw------- 1 root root      551250 Apr 22 02:06 max_cite_inputs.txt
-rw------- 1 root root     5723550 Apr 22 02:06 max_multi_inputs.txt
-rw------- 1 root root      234920 Apr 22 02:06 metadata_cite_day_2_donor_27678.csv
-rw------- 1 root root     9770334 Apr 22 02:06 metadata.csv
-rw------- 1 root root      551250 Apr 22 02:06 min_cite_inputs.txt
-rw------- 1 root root     5723550 Apr 22 02:06 min_multi_inputs.txt
-rw------- 1 root root   843563244 Apr 22 02:06 sample_submission.csv
-rw------- 1 root root   307964530 Apr 22 02:07 test_cite_inputs_day_2_donor_27678.h5
-rw------- 1 root root  1704565845 Apr 22 02:07 test_cite_inputs.h5
-rw------- 1 root root  6473530657 Apr 22 02:09 test_multi_inputs.h5
-rw------- 1 root root  2498128492 Apr 22 02:09 train_cite_inputs.h5
-rw------- 1 root root    38539123 Apr 22 02:09 train_cite_targets.h5
-rw------- 1 root root 11334840656 Apr 22 02:13 train_multi_inputs.

#Installation of required software packages

In [7]:
!pip install h5py==3.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Ref: https://docs.h5py.org/en/stable/mpi.html
#check whether parallel version of h5py is availiable
!h5cc -showconfig

In [8]:
!pip install hdf5plugin~=2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 3.5 MB/s eta 0:00:00


#HDF5 handling common code

In [10]:
import h5py
import hdf5plugin #without importing this, decompression will not happen by h5py
def get_hdf5_dataset_value_key(hdf5_file, debug = 0):
    groups = []
    def node_visit(name):
        groups.append(name)
    
    hdf5_file.visit(node_visit)
    if debug>0: print(hdf5_file, groups)
    
    for g in groups:
        shape = hdf5_file[g].shape if isinstance(hdf5_file[g], h5py._hl.dataset.Dataset) else None
        if debug>0: print(g, type(hdf5_file[g]), shape)
        if (not shape is None) and (len(shape) == 2):
            return g
    
    return None

def get_hdf5_dataset_with_specific_shape(hdf5_file, size, debug = 0):
    groups = []
    def node_visit(name):
        groups.append(name)
    
    hdf5_file.visit(node_visit)
    if debug>0: print(hdf5_file, groups)
    
    for g in groups:
        shape = hdf5_file[g].shape if isinstance(hdf5_file[g], h5py._hl.dataset.Dataset) else None
        if debug>0: print(g, type(hdf5_file[g]), shape)
        if (not shape is None) and (len(shape) == 1) and (shape[0] == size):
            return g
    
    return None

def get_hdf5_info(hdf5_file):
    print('root-group file-object name:', hdf5_file.name)
    def print_keys(gr, level):
        keys = list(gr.keys())
        for k in keys:
            
            if isinstance(gr[k], h5py._hl.group.Group):
                print('->'*level, k, gr[k])
                print_keys(gr[k], level + 1)
            elif isinstance(gr[k], h5py._hl.dataset.Dataset):
                print('->'*level, k, gr[k], 'dtype=', gr[k].dtype , 'size=', gr[k].size, 'nbytes=', gr[k].nbytes, 
                      'maxshape=', gr[k].maxshape, 'chunks=', gr[k].chunks)

    print_keys(hdf5_file, 1)



In [11]:
import h5py
import hdf5plugin #without importing this, decompression will not happen by h5py
print('============= TRAIN MULTI INPUT ====================')
train_multi_input_file = h5py.File('/mnt/train_multi_inputs.h5') # HDF5 file
get_hdf5_info(train_multi_input_file)
train_multi_input_file.close()
del train_multi_input_file
print('============= TEST MULTI INPUT ====================')
test_multi_input_file = h5py.File('/mnt/test_multi_inputs.h5') # HDF5 file
get_hdf5_info(test_multi_input_file)
test_multi_input_file.close()
del test_multi_input_file

============= TRAIN MULTI INPUT ====================
root-group file-object name: /
-> train_multi_inputs <HDF5 group "/train_multi_inputs" (4 members)>
->-> axis0 <HDF5 dataset "axis0": shape (228942,), type "|S26"> dtype= |S26 size= 228942 nbytes= 5952492 maxshape= (228942,) chunks= (2520,)
->-> axis1 <HDF5 dataset "axis1": shape (105942,), type "|S12"> dtype= |S12 size= 105942 nbytes= 1271304 maxshape= (105942,) chunks= (5461,)
->-> block0_items <HDF5 dataset "block0_items": shape (228942,), type "|S26"> dtype= |S26 size= 228942 nbytes= 5952492 maxshape= (228942,) chunks= (2520,)
->-> block0_values <HDF5 dataset "block0_values": shape (105942, 228942), type "<f4"> dtype= float32 size= 24254573364 nbytes= 97018293456 maxshape= (105942, 228942) chunks= (1, 228942)
============= TEST MULTI INPUT ====================
root-group file-object name: /
-> test_multi_inputs <HDF5 group "/test_multi_inputs" (4 members)>
->-> axis0 <HDF5 dataset "axis0": shape (228942,), type "|S26"> dtype= |S2

In [12]:
import h5py
import hdf5plugin #without importing this, decompression will not happen by h5py
train_mult_input_file = h5py.File('/mnt/train_multi_inputs.h5') # HDF5 file
hdf5_input_key = get_hdf5_dataset_value_key(train_mult_input_file, debug=1)

<HDF5 file "train_multi_inputs.h5" (mode r)> ['train_multi_inputs', 'train_multi_inputs/axis0', 'train_multi_inputs/axis1', 'train_multi_inputs/block0_items', 'train_multi_inputs/block0_values']
train_multi_inputs <class 'h5py._hl.group.Group'> None
train_multi_inputs/axis0 <class 'h5py._hl.dataset.Dataset'> (228942,)
train_multi_inputs/axis1 <class 'h5py._hl.dataset.Dataset'> (105942,)
train_multi_inputs/block0_items <class 'h5py._hl.dataset.Dataset'> (228942,)
train_multi_inputs/block0_values <class 'h5py._hl.dataset.Dataset'> (105942, 228942)


In [ ]:
hdf5_col_name_key = get_hdf5_dataset_with_specific_shape(train_mult_input_file, 228942, debug=1)
cols = train_mult_input_file[hdf5_col_name_key]
print(cols.shape)
from tqdm import tqdm
col_name = []
for c_id in tqdm(range(cols.shape[0])):
    col_name.append(str(cols[c_id], 'UTF-8'))

*   https://luis-sena.medium.com/sharing-big-numpy-arrays-across-python-processes-abf0dc2a0ab2 (why ray with shared object store is best sol)
*   Ref: https://towardsdatascience.com/histogram-on-function-space-4a710241f026
*   Ref: https://stackoverflow.com/questions/71844846/is-there-a-faster-way-to-get-correlation-coefficents (fast corr-coef)



# Global min and max determination of the raw-inputs (will be used for min-max normalization of the data)

In [104]:
%%writefile rawInputDataset.py

#read: https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac
#The above medium article explains why task related class (or the function which is being called from multiprocessor)
# has to be defined in separate python class for jupyter notebook

from h5py import File
from h5py import _hl
import hdf5plugin #without importing this, decompression will not happen by h5py
from numpy import zeros, s_
from torch import cuda, device, from_numpy, transpose
from torch import min as torch_min
from torch import max as torch_max
from subprocess import check_output

class RawInputDataset:
    
    #ref: https://stackoverflow.com/questions/59567226/how-to-programmatically-determine-available-gpu-memory-with-tensorflow
    def __get_gpu_memory(self):
        command = "nvidia-smi --query-gpu=memory.free --format=csv"
        try:
            memory_free_info = check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
            memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
            return memory_free_values[0] * 1024 * 1024 # memory_free_values[0] is in MB, thus converting into bytes
        except Exception as e:
            return -1

    def __get_hdf5_dataset_value_key(self):
        groups = []
        def node_visit(name):
            groups.append(name)
    
        self.hdf5_input.visit(node_visit)
        if self.debug>0: self.logger.info('%s %s', self.hdf5_input, groups)
    
        for g in groups:
            shape = self.hdf5_input[g].shape if isinstance(self.hdf5_input[g], _hl.dataset.Dataset) else None
            if self.debug>0: self.logger.info('group-key:%s, type:%s, shape:%s', g, type(self.hdf5_input[g]), shape)
            if (not shape is None) and (len(shape) == 2):
                return g
    
        return None

    def __init__(self, hdf5_input_path, batch_size, debug, logger):

        self.inited = False
        if debug > 0: logger.info('pid=%s hdf5_input_path:%s batch_size:%s debug:%s', os.getpid(), hdf5_input_path, batch_size, debug)

        assert batch_size >= 1

        self.hdf5_input_path = hdf5_input_path
        self.batch_size=batch_size
        self.debug = debug
        self.logger = logger
        
        #self.hdf5_input = File(self.hdf5_input_path, 'r', driver='stdio')
        self.hdf5_input = File(self.hdf5_input_path, 'r')
        hdf5_input_key = self.__get_hdf5_dataset_value_key()
        self.hdf5_dataset = self.hdf5_input[hdf5_input_key]
        self.len = self.hdf5_dataset.shape[0]

        self.cuda_device = device("cuda:0" if cuda.is_available and (self.__get_gpu_memory() > 0) else "cpu")
        self.input = zeros((batch_size,self.hdf5_dataset.shape[1]), dtype=self.hdf5_dataset.dtype)
        self.stat_id_consumed = []
        if debug > 0: self.logger.info('hdf5 file: %s hdf5 group: %s hdf5 dataset: %s id=%s', self.hdf5_input, hdf5_input_key, self.hdf5_dataset, id(self.hdf5_input))
        if debug > 1: self.logger.debug('torch-device:%s self.input:%s', self.cuda_device, self.input.shape)
        self.inited =True

    def is_inited(self):
        return self.inited
    
    def __len__(self): return self.len   

    def __getitem__(self, row):
        try:
            assert row < self.len
            input = self.hdf5_dataset[row]
            if self.debug>0:
                self.stat_id_consumed.append(row)
            if self.debug > 3: self.logger.debug('type of input=%s shape=%s', type(input) , input.shape)
            return from_numpy(input).detach().to(self.cuda_device)
        except Exception as e:
            self.logger.exception('Exception occurred in __getitem__()')
            
        return None

    def get_batch(self, starting_row):
        try:
            assert starting_row < self.len
            end_row = min(starting_row + self.batch_size, self.len)            
            #input = self.hdf5_dataset[starting_row:end_row]
            input = self.input
            if input.shape[0] != (end_row - starting_row): #will happen for the last batch
                input = zeros(((end_row - starting_row),self.hdf5_dataset.shape[1]), dtype=self.hdf5_dataset.dtype)
            self.hdf5_dataset.read_direct(input, source_sel=s_[starting_row:end_row,:], dest_sel=None)
            if self.debug>0:
                self.stat_id_consumed.extend(range(starting_row, end_row))
            if self.debug > 3: self.logger.debug('type of input=%s shape=%s', type(input) , input.shape)
            return from_numpy(input).to(self.cuda_device) #.detach()
        except Exception as e:
            self.logger.exception('Exception occurred in get_batch()')
            
        return None

    def find_min_max_on_batch(self, starting_row):
        try:
            data = self.get_batch(starting_row)
            if data is None:
                return (None, None)
            local_min = torch_min(data, dim=0)[0] #we have to find min for each col (so reduction of dim=0)
            local_max = torch_max(data, dim=0)[0] #we have to find max for each col (so reduction of dim=0)  
            #local_min = torch_min(transpose(data, 0, 1), dim=1)[0] #we have to find min for each row after transpose (so reduction of dim=1)
            #local_max = torch_max(transpose(data, 0, 1), dim=1)[0] #we have to find max for each row after transpose (so reduction of dim=1)  
            return (local_min, local_max)
        except Exception as e:
            self.logger.exception('Exception occurred in find_min_max_on_batch()')
        
        return (None, None)

    def reset_stat(self):
        if self.debug <= 0:
           return
        self.stat_id_consumed.clear()    

    def __del__(self):
        self.hdf5_input.close()


Overwriting rawInputDataset.py


In [32]:
%%writefile custom_mp.py

#read: https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac
#The above medium article explains why Dataset related class (or the function which is being called from multiprocessor)
# has to be defined in separate python class for jupyter notebook

def my_custom_init(hdf5_input_file_path, batch_size, debug):
    from rawInputDataset import RawInputDataset
    from multiprocessing import current_process
    from threading import current_thread
    import logging
    from os import getpid

    logging.basicConfig(filename=f'my_log-{getpid()}.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')
    
    global logger
    logger=logging.getLogger(f'child-worker-{getpid()}')
    logger.setLevel(logging.DEBUG) 

    try: 
       # declare global variable (#https://superfastpython.com/multiprocessing-pool-initializer/#Example_of_Accessing_an_Initialized_Variable_in_a_Worker)
       global my_rawInputDataset
       # assign the global variable
       my_rawInputDataset = RawInputDataset(hdf5_input_file_path, batch_size, debug, logger)
       # get the current process
       process = current_process()
       # get the current thread
       thread = current_thread()
       # report a message
       logger.info('Initializing worker pid=%s process=%s, thread=%s, with=%s', getpid(), process.name, thread.name, my_rawInputDataset)
    except Exception as e:
        logger.exception('Exception occurred in my_custom_init()')


def my_custom_min_max_finding_job(starting_row):
    try:
        global my_rawInputDataset  
        return my_rawInputDataset.find_min_max_on_batch(starting_row)
    except Exception as e:
        global logger
        logger.exception('Exception occurred in my_custom_min_max_finding_job()')
    
    return (None, None)


Overwriting custom_mp.py


In [15]:
#https://stackoverflow.com/questions/2918898/prevent-python-from-caching-the-imported-modules
#we have to ensure 'rawInputDataset' module is reloaded everytime (helps us to continously change source code of rawInputDataset.py file)
%load_ext autoreload

In [16]:
%autoreload 2

In [ ]:
!python3 -m pip install memray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 50.6 MB/s eta 0:00:00


In [ ]:
%load_ext memray

In [106]:
!rm -f *.log

In [105]:
#%%memray_flamegraph --trace-python-allocators --follow-fork --native --leaks 

def find_min_max():
    
    try:
        hdf5_input_file_path = '/mnt/train_multi_inputs.h5'
        hdf5_input = h5py_File(hdf5_input_file_path, 'r')
        hdf5_input_key = get_hdf5_dataset_value_key(hdf5_input)
        data_len = hdf5_input[hdf5_input_key].shape[0]
        first_elem = hdf5_input[hdf5_input_key][0]
        elem_size_in_bytes = first_elem.size * first_elem.itemsize
        print('first_elem.size:', first_elem.size, 'first_elem.itemsize:', first_elem.itemsize, 'elem_size_in_bytes:', elem_size_in_bytes)
        hdf5_input.close()
        del hdf5_input

        optimal_batch_size = floor((1024 * 1024 * 1024) / elem_size_in_bytes) #max 1GB of numpy array 
        #optimal_batch_size = floor((50 * 1024 * 1024) / elem_size_in_bytes) #max 50MB of numpy array 
        print('optimal_batch_size:', optimal_batch_size)

        while(collect() > 0): pass #clean the memory as much as possible

        DEBUG = 0

        work_arg = []
        for s_row in range(0, data_len, optimal_batch_size):
        #for s_row in range(0, 6 * optimal_batch_size, optimal_batch_size):
            work_arg.append(s_row)
        print("total task required:", len(work_arg))
        min_t = None
        max_t = None
        with  get_context('spawn').Pool(os.cpu_count() * 2, initializer=my_custom_init, initargs=(hdf5_input_file_path,optimal_batch_size,DEBUG )) as p:
            #For very long iterables using a large value for chunksize can make the job complete much faster than using the default value of 1 
            #(refer: https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool.imap_unordered)
            result = p.imap_unordered(my_custom_min_max_finding_job, work_arg, chunksize=1)
            
            for r in tqdm(result):
                if (r[0] is None) or (r[1] is None):
                    print("None result obtained!!!", flush=True)
                    p.terminate()
                    break
                if not (min_t is None):
                    min_t = minimum(min_t, r[0])
                    max_t = maximum(max_t, r[1])
                else:
                    min_t = r[0]
                    max_t = r[1]

            # close the process pool
            p.close()
            # wait for all tasks to complete
            p.join()
            
        print('max.shape:', max_t.shape, 'min.shape:', min_t.shape)
        print('max_t', max_t, 'min_t:', min_t) #[10.1655, 11.7670,  9.6392,  ..., 10.8656, 13.2418, 12.2571]
        return max_t, min_t
    except Exception as e:
        print(e)
        print_exc()
    

if __name__ == '__main__':

    from math import floor
    from traceback import print_exc
    from multiprocessing import get_context
    from custom_mp import my_custom_init, my_custom_min_max_finding_job
    from gc import collect, get_stats
    from os import getpid
    from h5py import File as h5py_File
    from torch import minimum, maximum
    from tqdm import tqdm
    
    print('MAIN pid=', getpid())
    find_min_max()
    collect()
    print(get_stats())

MAIN pid= 246
first_elem.size: 228942 first_elem.itemsize: 4 elem_size_in_bytes: 915768
optimal_batch_size: 1172
total task required: 91


91it [01:48,  1.20s/it]


max.shape: torch.Size([228942]) min.shape: torch.Size([228942])
max_t tensor([10.1655, 11.7670,  9.6392,  ..., 10.8656, 13.2418, 12.2571],
       device='cuda:0') min_t: tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
[{'collections': 643, 'collected': 2725, 'uncollectable': 0}, {'collections': 51, 'collected': 646, 'uncollectable': 0}, {'collections': 75, 'collected': 2669, 'uncollectable': 0}]


In [ ]:
np.savetxt('/content/drive/MyDrive/colab_exp_result/kaggle_data/max_multi_inputs.txt', max_m.numpy())
np.savetxt('/content/drive/MyDrive/colab_exp_result/kaggle_data/min_multi_inputs.txt', min_m.numpy())

In stead of calculating min max of the input, we can read it everytime from a saved location. This will save time in terms of rerunning the min-max finding algorithm.

In [ ]:
max_multi = np.float32(np.loadtxt('/content/drive/MyDrive/colab_exp_result/kaggle_data/max_multi_inputs.txt'))
min_multi = np.float32(np.loadtxt('/content/drive/MyDrive/colab_exp_result/kaggle_data/min_multi_inputs.txt'))
print(max_multi.shape)
print(min_multi.shape)